# AIDM7360 Group Project
# NBA Report Management System

In [9]:
import sqlite3
from sqlite3 import Error

In [10]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn

def showOptions():
    """ Show the options """
    choices = ["Report performance of Lakers and opponent in one game(by date)", "Report James's performance in one game(by date)",
               "Compare James's performance in one game with that in the games which Lakers won or lost(by date)",
               "Compare the performance of James in one game with that of top players in Bucks & Heat(by date)", 
               "Add a game of Lakers", "Delete a game of Lakers", "Exit"]
    print("---------------------------")
    print("Please choose following option:")
    print("1. ", choices[0])
    print("2. ", choices[1])
    print("3. ", choices[2])
    print("4. ", choices[3])
    print("5. ", choices[4])
    print("6. ", choices[5])
    print("7. ", choices[6])

## For Chioce '1': Performance of Lakers and opponent in one game

In [1]:
def TeamsPerformance(conn):
    date = input("Please input date of a game(For example: 2020/1/2):")
    date = date.strip()
    with conn:
        team = "SELECT * FROM LAKERS WHERE DATE = ?"
        cur = conn.cursor()
        cur.execute(team, [date])
        game = cur.fetchone()
        if game is None:
            print("No such game.")
        else:
            if game[2] == "W":
                result = "won"
            elif game[2] == "L":
                result = "lost"
            avgteam = "SELECT PPG FROM TEAMS WHERE TEAM = ?"
            cur = conn.cursor()
            cur.execute(avgteam, ["LAL"])
            avgla = cur.fetchone()
            if game[3] > avgla[0]:
                highlow = "higher than"
                diff = '%.1f'%(abs(game[3] - avgla[0]))
            elif game[3] < avgla[0]:
                highlow = "lower than"
                diff = '%.1f'%(abs(game[3] - avgla[0]))
            else:
                highlow = "equal to"
                diff = ""
            
            opp = game[1]
            cur = conn.cursor()
            cur.execute(avgteam, [opp])
            avgte = cur.fetchone()
            if game[4] > avgte[0]:
                highlowot = "higher than"
                diffot = '%.1f'%(abs(game[4] - avgte[0]))
            elif game[4] < avgte[0]:
                highlowot = "lower than"
                diffot = '%.1f'%(abs(game[4] - avgte[0]))
            else:
                highlowot = "equal to"
                diffot = ""
        
            lakers = "In {} match in {}, Lakers {} the game, and got {} points, {} {} its average points in regular game."
            teams = "The opponent {} got {} points, {} {} its average points in regular game."
        
            print(lakers.format(date,game[5],result,game[3],diff,highlow))
            print(teams.format(opp,game[4],diffot,highlowot))          

# For Chioce '2': James's performance in one game

In [2]:
def JamesPerformance(conn):
    date = input('Please input date of a game(For example: 2020/1/2):')
    date = date.strip()
    with conn:

        james = "SELECT _MIN,PTS,REB,FG,AST,TOV,LOCATION FROM JAMES,LAKERS WHERE JAMES.DATE=LAKERS.DATE AND JAMES.DATE =?"
        cur = conn.cursor()
        cur.execute(james,[date])
        jdate = cur.fetchone()
        if jdate is None:
            print("James did not participate in this game.")
        else:
            _min = jdate[0]
            pts = jdate[1]
            reb = jdate[2]
            fg = jdate[3]
            ast = jdate[4]
            tov = jdate[5]
            loca = jdate[6]
    jamesper = "In {} match in {}, James played {} minutes, scored {} points, and had {} rebounds and {} assists.\
    \nHis field goal percentage and turnovers were {}, {} respectively."
    print(jamesper.format(date,loca,_min,pts,reb,ast,fg,tov))      

## For Chioce '3': Compare James's performance in one game with that in the games which Lakers won or lost

In [4]:
def SelfCompare(conn):
    date = input('Please input date of a game(For example: 2020/1/2):')
    date = date.strip()
    with conn:

        james = "SELECT _MIN,PTS,REB,FG,AST,TOV,LOCATION FROM JAMES,LAKERS WHERE JAMES.DATE=LAKERS.DATE AND JAMES.DATE =?"
        cur = conn.cursor()
        cur.execute(james,[date])
        jdate = cur.fetchone()
        if jdate is None:
            print("James did not participate in this game.")
        else:
            _min = jdate[0]
            pts = jdate[1]
            reb = jdate[2]
            fg = jdate[3]
            ast = jdate[4]
            tov = jdate[5]
            loca = jdate[6]
        mpts = "SELECT MAX(JAMES.PTS),AVG(JAMES.PTS),MIN(JAMES.PTS) FROM JAMES,LAKERS WHERE JAMES.DATE=LAKERS.DATE AND LAKERS.RESULT=?"
        mast = "SELECT MAX(JAMES.AST),AVG(JAMES.AST),MIN(JAMES.AST) FROM JAMES,LAKERS WHERE JAMES.DATE=LAKERS.DATE AND LAKERS.RESULT=?"
        mtov = "SELECT MAX(JAMES.TOV),AVG(JAMES.TOV),MIN(JAMES.TOV) FROM JAMES,LAKERS WHERE JAMES.DATE=LAKERS.DATE AND LAKERS.RESULT=?"

        coun = "SELECT COUNT(*) FROM LAKERS WHERE RESULT=?"
        
        jamesasp=["points","assists","turnovers"]
        resu=["W","L"]
        
        cur = conn.cursor()
        cur.execute(coun,[resu[0]])
        counw = cur.fetchone()
        cur = conn.cursor()
        cur.execute(coun,[resu[1]])
        counl = cur.fetchone()
        
        cur = conn.cursor()
        cur.execute(mpts,[resu[0]])
        mwpts = cur.fetchall()
        cur = conn.cursor()
        cur.execute(mpts,[resu[1]])
        mlpts = cur.fetchall()
        cur = conn.cursor()
        cur.execute(mast,[resu[0]])
        mwast = cur.fetchall()
        cur = conn.cursor()
        cur.execute(mast,[resu[1]])
        mlast = cur.fetchall()
        cur = conn.cursor()
        cur.execute(mtov,[resu[0]])
        mwtov = cur.fetchall()
        cur = conn.cursor()
        cur.execute(mtov,[resu[1]])
        mltov = cur.fetchall()
        
        maxminw = mwpts+mwast+mwtov
        maxminl = mlpts+mlast+mltov
               
        print("Here is a comparison between James's performance in this game and that in the games which Lakers won or lost.")
        reg = "In the 2019-2020 regular season, The Lakers have won {} games and lost {} games this season."
        wl = "{}. When the Lakers won the game, the highest, the average and the lowest {} were {}, {}, {} respectively.\
        \nWhen the Lakers lost the game, the highest, the average and the lowest {} were {}, {}, {} respectively."
        print(reg.format(counw[0],counl[0]))
        
        for i in range(0,3):
            print(wl.format(i+1,jamesasp[i],maxminw[i][0],'%.1f'%(maxminw[i][1]),maxminw[i][2],\
                            jamesasp[i],maxminl[i][0],'%.1f'%(maxminl[i][1]),maxminl[i][2]))     

## For Chioce '4': Compare the performance of James in one game with that of top players in Bucks and Heat

In [5]:
def OthersCompare(conn,team):
    date = input('Please input date of a game(For example: 2020/1/2):')
    date = date.strip()
    with conn:
        
        james = "SELECT _MIN,PTS,REB,FG,AST,TOV,LOCATION FROM JAMES,LAKERS WHERE JAMES.DATE=LAKERS.DATE AND JAMES.DATE =?"
        cur = conn.cursor()
        cur.execute(james,[date])
        jdate = cur.fetchone()
        if jdate is None:
            print("James did not participate in this game.")
        else:
            _min = jdate[0]
            pts = jdate[1]
            reb = jdate[2]
            fg = jdate[3]
            ast = jdate[4]
            tov = jdate[5]
            loca = jdate[6]
            
        ptss = "SELECT PLAYER,PTS FROM PLAYERS WHERE TEAM =? AND PTS >=(SELECT MAX(PTS) FROM PLAYERS WHERE TEAM =?)"
        fgs = "SELECT PLAYER,FG FROM PLAYERS WHERE TEAM =? AND FG >=(SELECT MAX(FG) FROM PLAYERS WHERE TEAM =?)"
        rebs = "SELECT PLAYER,REB FROM PLAYERS P1 WHERE NOT EXISTS(SELECT * FROM PLAYERS P2 WHERE P1.REB<P2.REB AND P2.TEAM=?) AND P1.TEAM=?"
    
        cur = conn.cursor()
        cur.execute(ptss,(team,team))
        ptscom = cur.fetchall()
        cur = conn.cursor()
        cur.execute(fgs,(team,team))
        fgcom = cur.fetchall()
        cur = conn.cursor()
        cur.execute(rebs,(team,team))
        rebcom = cur.fetchall()
        
    c = "{}. The {} player {} ranked first in average {} in the regular season, and his average {} was {}, {} {} James's {} in this game."
    cla = ["points","field goal percentage","rebounds"]
    com = (ptscom,fgcom,rebcom)
    dat = [pts,fg,reb]
    
    for i in range(0,3):
        
        if com[i][0][1]>dat[i]:
            diff = format(abs(com[i][0][1]-dat[i]), '.1f')
            
            highlow = "higher than"
        elif com[i][0][1]<dat[i]:
            diff = format(abs(com[i][0][1]-dat[i]), '.1f')
            highlow = "lower than"
        else:
            diff = ""
            highlow = "equal to"
        print(c.format(i+1,team,com[i][0][0],cla[i],cla[i],com[i][0][1],diff,highlow,cla[i]))

## For Choice '5': Add a game of Lakers

In [6]:
def AddGame(conn):

    print("Please input the date, opponent, result, point_Lakers, point_opp, location.")
    add = input("(For example: 2020/1/1, LAC, W, 110, 100, Staples Center):")
    values = add.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    with conn:
        sql = "INSERT INTO LAKERS VALUES(?, ?, ?, ?, ?, ?)"
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1], values[2], int(values[3]), int(values[4]), values[5]))
        print("Succeed to add game!!!")
        return cur.lastrowid

## For Choice '6': Delete a game of Lakers

In [7]:
def DeleteGame(conn):

    date = input("Please input the date to delete(For example: 2020/1/1): ")
    date = date.strip()
    with conn:
        
        sql = "DELETE FROM LAKERS WHERE DATE =?"
        cur = conn.cursor()
        cur.execute(sql, [date])
        conn.commit()
        print("Succeed to delete game: ", date)

## Finally, let's get started to report

In [ ]:
"""main"""
database = "NBA.db"
conn = create_connection(database)
while True:
    showOptions()
    choice = input('Please enter your option: ')
    # with conn:
    if choice == '7':
        conn.close() # close the connection
        break
    elif choice == '1':
        TeamsPerformance(conn)
    elif choice == '2':
        JamesPerformance(conn)
    elif choice == '3':
        SelfCompare(conn)
    elif choice == '4':
        print("In the 2019-2020 season, the Bucks and Heat were also very competitive teams.")
        print("Here is a comparison between James's performance in this game and the average performance of other top players in the regular season.")
        print("Compared with top players in Bucks:")    
        OthersCompare(conn,"MIL")
        print("Compared with top players in Heat:")
        OthersCompare(conn,"MIA") 
    elif choice == '5':
        AddGame(conn)
    elif choice == '6':
        DeleteGame(conn)
    else:
        print("This option is not available")
        continue

---------------------------
Please choose following option:
1.  Report performance of Lakers and opponent in one game(by date)
2.  Report James's performance in one game(by date)
3.  Compare James's performance in one game with that in the games which Lakers won or lost(by date)
4.  Compare the performance of James in one game with that of top players in Bucks & Heat(by date)
5.  Add a game of Lakers
6.  Delete a game of Lakers
7.  Exit


Please enter your option:  1
Please input date of a game(For example: 2020/1/2): 2019/12/29


In 2019/12/29 match in Moda Center, Lakers won the game, and got 128 points, 14.6 higher than its average points in regular game.
The opponent POR got 120 points, 5.0 higher than its average points in regular game.
---------------------------
Please choose following option:
1.  Report performance of Lakers and opponent in one game(by date)
2.  Report James's performance in one game(by date)
3.  Compare James's performance in one game with that in the games which Lakers won or lost(by date)
4.  Compare the performance of James in one game with that of top players in Bucks & Heat(by date)
5.  Add a game of Lakers
6.  Delete a game of Lakers
7.  Exit


Please enter your option:  2
Please input date of a game(For example: 2020/1/2): 2019/12/29


In 2019/12/29 match in Moda Center, James played 37 minutes, scored 21 points, and had 7 rebounds and 16 assists.    
His field goal percentage and turnovers were 46.7, 1 respectively.
---------------------------
Please choose following option:
1.  Report performance of Lakers and opponent in one game(by date)
2.  Report James's performance in one game(by date)
3.  Compare James's performance in one game with that in the games which Lakers won or lost(by date)
4.  Compare the performance of James in one game with that of top players in Bucks & Heat(by date)
5.  Add a game of Lakers
6.  Delete a game of Lakers
7.  Exit


Please enter your option:  3
Please input date of a game(For example: 2020/1/2): 2019/12/29


Here is a comparison between James's performance in this game and that in the games which Lakers won or lost.
In the 2019-2020 regular season, The Lakers have won 52 games and lost 19 games this season.
1. When the Lakers won the game, the highest, the average and the lowest points were 40, 26.8, 13 respectively.        
When the Lakers lost the game, the highest, the average and the lowest points were 31, 21.1, 13 respectively.
2. When the Lakers won the game, the highest, the average and the lowest assists were 16, 10.4, 4 respectively.        
When the Lakers lost the game, the highest, the average and the lowest assists were 19, 9.7, 4 respectively.
3. When the Lakers won the game, the highest, the average and the lowest turnovers were 8, 3.9, 0 respectively.        
When the Lakers lost the game, the highest, the average and the lowest turnovers were 8, 4.0, 0 respectively.
---------------------------
Please choose following option:
1.  Report performance of Lakers and opponent in